# Belief Revision Assignment

02180 Introduction to Artificial Intelligence

You are asked to implement a belief revision agent. By default, the project should run through
the following sequence of stages:
1. design and implementation of belief base;
2. design and implementation of a method for checking logical entailment (e.g., resolution-based), you should implement it yourself, without using any existing packages;
3. implementation of contraction of belief base (based on a priority order on alphas in the belief base);
4. implementation of expansion of belief base.
The output should be the resulting/new belief base.

## TODO

- [ ] Method to check logical entailment
- [ ] Contraction of belief base
- [ ] Expansion of belief base
- [ ] Use AGM postulates (lecture 11) to test your algorithm

In [9]:
from sympy import to_cnf
from sympy import Or, And, Not

In [10]:
def disjuncts(clause):
    #Adapted from tdiam's GitHub repository "belief-revision-engine"
    return dissociate(Or, [clause])

def conjuncts(clause):
    #Adapted from tdiam's GitHub repository "belief-revision-engine"
    return dissociate(And, [clause])

def dissociate(op, args):
    #Adapted from tdiam's GitHub repository "belief-revision-engine"
    result = []

    def collect(subargs):
        for arg in subargs:
            if isinstance(arg, op):
                collect(arg.args)
            else:
                result.append(arg)

    collect(args)
    return result

def make_clause(args):
    args = dissociate(Or, args)
    # if there are no arguments, return False
    if len(args) == 0:
        return False
    elif len(args) == 1:
        return args[0]
    else:
        return Or(*args)

In [11]:
from sympy import Equality


class BeliefBase:
    def __init__(self):
        self.beliefs = []
        self.belief_base = []

    def add_belief(self, belief) -> None:
        """Add a belief to the belief base, keeping the belief base in CNF form"""
        belief = to_cnf(belief)
        #if belief.is_Atom:
         #   belief = to_cnf(And(belief, belief))
        self.belief_base.append(belief)

    def pl_resolution(self, alpha):
        """
        Implements the pl-resolution algorithm for clauses resolution.

        Args:
            alpha (sympy.Expr): A SymPy sentence representing the query.

        Returns:
            bool: True if the query is entailed by the knowledge base, False otherwise.
        """
        # Split each clause in the base by the "And" symbol, turning them into a list of disjunctions
        clauses = []
        for clause in self.belief_base:
            clauses += conjuncts(clause)
        # Add negation of alpha as clause
        alpha_lit = conjuncts(to_cnf(Not(alpha)))
        for clause in alpha_lit:
            clauses.append(clause)

        new_clauses = set()  
        while True:
            resolvents = set()
            for ci in clauses:
                for cj in clauses:
                    if ci != cj:
                        resolvents |= self.pl_resolve(ci, cj)
                if any(clause == False for clause in resolvents):  # Empty clause found
                    return True

            if resolvents.issubset(clauses):  # No new clauses derived
                return False

            new_clauses |= resolvents
            clauses.extend(new_clauses)

    def pl_resolve(self, clause1, clause2):
        """
        Performs the resolution of two clauses.

        Args:
            clause1 (sympy.Expr): A SymPy sentence representing a clause.
            clause2 (sympy.Expr): A SymPy sentence representing a clause.

        Returns:
            set: The set of caluses resulting from the resolution operation.
        """
        clause1 = disjuncts(clause1)
        clause2 = disjuncts(clause2)
        resolvents = set()

        for ci in clause1:
            for cj in clause2:
                if ci == Not(cj) or cj == Not(ci):
                    # remove ci and cj from resolved clause
                    res = (set(clause1) - {ci}).union(set(clause2) - {cj})
                    resolvents.add(make_clause(res))
        return resolvents
                    

    def __str__(self):
        return str(self.belief_base)

In [12]:
# Create an instance of BeliefBase
kb = BeliefBase()

# Add a belief to the belief base
kb.add_belief("q & p & Not(r)")
# belief_base.add_belief("r | s")
print(f"After adding: {kb}")

# Check if a belief is entailed by the belief base
alpha = "q & Not(r)"
entails = kb.pl_resolution(alpha)
print(f"entails {alpha}: {entails}")

# The problem is that we have to feed the resolution function with a discunction form and a sentence
# We also have to adapt the transformation functions to how our belief base works

After adding: [p & q & ~r]
entails q & Not(r): True
